In [1]:
import elliot
elliot.__version__

'0.1'

In [2]:
from elliot.evaluation.metrics import *

In [3]:
from elliot.evaluation.relevance import Relevance

In [4]:
from types import SimpleNamespace

In [5]:
import pandas as pd
import numpy as np

In [6]:
df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

In [7]:
df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [8]:
df_test = df_test[df_test.user_id.isin(df_pred.user_id)]

In [9]:
recommendations = {}

for user, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.score):

    if user in recommendations:
        recommendations[user] += [(item_id, score)]
    else:
        recommendations[user] = [(item_id, score)] 

In [10]:
gt_recommendations = {}

for user, item_id, score in zip(df_test.user_id, df_test.item_id, df_test.relevance):
    
#     if score > 0:

    if user in gt_recommendations:
        gt_recommendations[user][item_id] = int(score > 0)
    else:
        gt_recommendations[user] = {item_id: int(score > 0)}

In [11]:
Precision(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.057519209659714604

In [12]:
Recall(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.09632131280310642

In [13]:
MAP(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.07298532565140084

In [14]:
MRR(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.18632413351027097

In [15]:
HR(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.47457006952067327

In [16]:
NDCG(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.09326437896736814

In [17]:
gt_recommendations = {}

for user, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.relevance):
    if user in gt_recommendations:
        gt_recommendations[user][item_id] = score
    else:
        gt_recommendations[user] = {item_id: score}

In [18]:
AUC(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff=None,
              num_items=19,
              data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
          )
).eval()

0.6023404513183068

In [19]:
GAUC(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff=20,
              num_items=20,
              data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
          )
).eval()

0.6187365226601701

In [20]:
LAUC(
    recommendations, 
    SimpleNamespace(config_test=False), 
    None, 
    eval_objects = SimpleNamespace(
      relevance=Relevance(gt_recommendations, 0.5),
      cutoff = 20,
      num_items=19,
      data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
    )
).eval()

0.5966968658330363